## Building Machine Learning Classifiers: Building a basic Random Forest Model

### Read in & clean text

In [31]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [7]:
stopwords = stopwords.words('english')
ps = PorterStemmer()

data = pd.read_csv('data/SMSSpamCollection.tsv', sep='\t', names=['label', 'body_text'])
data.head()

,label,body_text
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!


In [8]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    
    return round(count/(len(text) - text.count(' ')), 3) * 100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    
    return text

In [9]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8327,8328,8329,8330,8331,8332,8333,8334,8335,8336
0,160,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
count_vect = CountVectorizer()
X_count = count_vect.fit_transform(data['body_text'])

X_count_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count_features.toarray())], axis=1)
X_count_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8700,8701,8702,8703,8704,8705,8706,8707,8708,8709
0,160,2.5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Explore RandomForestClassifier Attributes & Hyperparameters

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_n_features', '_estimator_type', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_set_oob_score', '_validate_X_predict', '_validate_data', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier()


### Explore RandomForestClassifier through Cross-Validation

In [12]:
from sklearn.model_selection import KFold, cross_val_score

In [13]:
rf = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
cross_val_score(rf, X_features, data['label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

array([0.97486535, 0.98025135, 0.97486535, 0.96675651, 0.97663971])

### Explore RandomForestClassifier through Holdout Set

In [14]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

print(X_train.shape)
print(X_test.shape)

(4454, 8339)
(1114, 8339)


In [16]:
rf_clf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf_clf.fit(X_train, y_train)

In [18]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[:10]

[(0.0452547042187419, 'body_len'),
 (0.03794158508981685, 2244),
 (0.03315080242531176, 2017),
 (0.031284657981457305, 5009),
 (0.028905673836786442, 3349),
 (0.019878621658875082, 7689),
 (0.015893989434109152, 7247),
 (0.015388019455455346, 6207),
 (0.01494195022471517, 7577),
 (0.014915659326730708, 8013)]

In [19]:
rf_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 20,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 50,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [24]:
y_pred = rf_model.predict(X_test)
precision, recall, f1_score, support = score(y_test, y_pred, pos_label='spam', average='binary')

In [26]:
print(f"Precision: {round(precision, 3)} / Recall:{round(recall, 3)} / Accuracy: {round((y_pred == y_test).sum()/len(y_test), 3)}")

Precision: 1.0 / Recall:0.553 / Accuracy: 0.939


### Exploring RandomForestClassifier with Grid-Search

In [28]:
def train_rf(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, f1_score, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print(f"Est: {n_est} / Depth: {depth} ------- Precision: {round(precision, 3)} / Recall:{round(recall, 3)} / Accuracy: {round((y_pred == y_test).sum()/len(y_test), 3)}")

In [29]:
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_rf(n_est, depth) 

Est: 10 / Depth: 10 ------- Precision: 1.0 / Recall:0.211 / Accuracy: 0.892
Est: 10 / Depth: 20 ------- Precision: 1.0 / Recall:0.599 / Accuracy: 0.945
Est: 10 / Depth: 30 ------- Precision: 1.0 / Recall:0.737 / Accuracy: 0.964
Est: 10 / Depth: None ------- Precision: 1.0 / Recall:0.829 / Accuracy: 0.977
Est: 50 / Depth: 10 ------- Precision: 1.0 / Recall:0.283 / Accuracy: 0.902
Est: 50 / Depth: 20 ------- Precision: 1.0 / Recall:0.612 / Accuracy: 0.947
Est: 50 / Depth: 30 ------- Precision: 1.0 / Recall:0.743 / Accuracy: 0.965
Est: 50 / Depth: None ------- Precision: 1.0 / Recall:0.842 / Accuracy: 0.978
Est: 100 / Depth: 10 ------- Precision: 1.0 / Recall:0.224 / Accuracy: 0.894
Est: 100 / Depth: 20 ------- Precision: 1.0 / Recall:0.592 / Accuracy: 0.944
Est: 100 / Depth: 30 ------- Precision: 1.0 / Recall:0.711 / Accuracy: 0.961
Est: 100 / Depth: None ------- Precision: 1.0 / Recall:0.849 / Accuracy: 0.979


In [27]:
from sklearn.model_selection import GridSearchCV

In [30]:
rf = RandomForestClassifier()
params = {
    'max_depth': [30, 60, 90, None],
    'n_estimators': [10, 150, 300]
    }

gs = GridSearchCV(rf, params, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_features, data['label'])

pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,26.630460,0.523501,0.385590,0.022644,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.976661,0.976661,0.975763,0.971249,0.972147,0.974496,0.002325,1
10,30.068597,0.935191,0.411046,0.008063,None,150,"{'max_depth': None, 'n_estimators': 150}",0.975763,0.980251,0.975763,0.966757,0.973046,0.974316,0.004431,2
11,54.612196,7.894773,0.486058,0.116651,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976661,0.976661,0.973070,0.967655,0.974843,0.973778,0.003339,3
8,51.371851,0.867502,0.538279,0.039003,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.976661,0.976661,0.974865,0.965858,0.973944,0.973598,0.004010,4
4,21.191827,0.123448,0.325324,0.012361,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.975763,0.976661,0.971275,0.964061,0.974843,0.972520,0.004608,5


In [ ]:
rf = RandomForestClassifier()
params = {
    'max_depth': [30, 60, 90, None],
    'n_estimators': [10, 150, 300]
    }

gs = GridSearchCV(rf, params, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_features, data['label'])

pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[:5]